In [1]:
import json
import pandas as pd
import geopandas as gpd
from datetime import datetime
import folium
from folium import plugins
import psycopg2
from sqlalchemy import create_engine

import plotly
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

"""
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
"""

'\nfrom bokeh.io import output_notebook, show, output_file\nfrom bokeh.plotting import figure\nfrom bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar\nfrom bokeh.palettes import brewer\n'

# Daten

## Postgresql

In [1]:
conn = psycopg2.connect(
    host="localhost",
    database="gd_db",
    user="postgres",
    password="password")

import config

cur = conn.cursor()
    
cur.execute('SELECT * FROM countries LEFT JOIN cases ON countries.country_region_code = cases."Country_code";')

countries_cases = cur.fetchall()

NameError: name 'psycopg2' is not defined

## Countries

In [24]:
#with open(r'GeospacialDBs_Data/countries.geo.json', 'r') as file:
#    countries = gpd.read_file(file)

#countries.drop(columns=['ISO_A3'],inplace=True)
#countries.columns=['country_region','Country_code','geometry']

with open(r'GeospacialDBs_Data/countries.geo.json') as json_file:
    countries = json.load(json_file)

print(countries["features"][200]["properties"])

{'ADMIN': 'Singapore', 'ISO_A3': 'SGP', 'ISO_A2': 'SG'}


## Corona cases

In [25]:
cases = pd.read_csv(r"GeospacialDBs_Data/WHO-COVID-19-global-data.csv",header=0)

cases.drop(columns=['WHO_region','Cumulative_cases','Cumulative_deaths'],inplace=True)

# Creating smaller dataset for testing

for i in range(0, len(cases)):
    cases.loc[i, "Date_reported"] = datetime.strptime(cases.loc[i, "Date_reported"], '%Y-%m-%d')

In [26]:
cases_small = cases[(datetime.strptime("2020-03-01", '%Y-%m-%d') <= cases.Date_reported) & (cases.Date_reported < datetime.strptime("2020-03-05", '%Y-%m-%d'))]
cases_small = cases_small.reset_index(drop=True)

for i in range(0, len(cases_small)):
    cases_small.loc[i, "Date_reported"] = cases_small.loc[i, "Date_reported"].strftime('%Y-%m-%d')
    
cases_small

,Date_reported,Country_code,Country,New_cases,New_deaths
0,2020-03-01,AF,Afghanistan,0,0
1,2020-03-02,AF,Afghanistan,0,0
2,2020-03-03,AF,Afghanistan,0,0
3,2020-03-04,AF,Afghanistan,0,0
4,2020-03-01,AL,Albania,0,0
...,...,...,...,...,...
943,2020-03-04,ZM,Zambia,0,0
944,2020-03-01,ZW,Zimbabwe,0,0
945,2020-03-02,ZW,Zimbabwe,0,0
946,2020-03-03,ZW,Zimbabwe,0,0


# Visualisierung

## Plotly

### Funktioniert bisher am besten

In [27]:
fig = px.choropleth_mapbox(cases_small, geojson=countries, locations='Country_code', color='New_cases',
                           featureidkey="properties.ISO_A2",
                           color_continuous_scale="Viridis",
                           #animation_frame="Date_reported",
                           range_color=(0, 12),
                           hover_name = 'Country',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Funktioniert weniger gut

In [ ]:
plotly.offline.init_notebook_mode()

In [5]:
data = [dict(type='choropleth',
             locations = cases['Country'].astype(str),
             z=cases['New_cases'].astype(float),
             locationmode='country names')]

"""
steps = []
for i in range(len(data)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data)],
                label='Day {}'.format(i))
    step['args'][1][i] = True
    steps.append(step)
    
sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]

layout = dict(geo=dict(scope='world',
                       projection={'type': 'mercator'}),
              sliders=sliders)

fig = dict(data=data, 
           layout=layout)
plotly.offline.iplot(fig)
"""

AttributeError: 'dict' object has no attribute 'show'

In [ ]:
scl = [[0.0, '#ffffff'],[0.2, '#b4a8ce'],[0.4, '#8573a9'],
       [0.6, '#7159a3'],[0.8, '#5732a1'],[1.0, '#2c0579']] # purples

data_slider = []
for date in cases['Date_reported'].unique():
    cases_slice =  cases[(cases['Date_reported']== date)]

    for col in cases_slice.columns:
        cases_slice[col] = cases_slice[col].astype(str)

    data_each_yr = dict(
                        type='choropleth',
                        locations = cases['Country_code'],
                        z=cases_slice['New_cases'].astype(float),
                        locationmode='country names',
                        colorscale = scl,
                        colorbar= {'title':'# cases'})

    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year')
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

layout = dict(title ='UFO Sightings by State Since 1998', geo=dict(scope='world',
                       projection={'type': 'mercator'}),
              sliders=sliders)

fig = dict(data=data_slider, layout=layout)

plotly.offline.iplot(fig)

## Folium

In [ ]:
m = folium.Map(
    location=[-59.1759, -11.6016],
    tiles="cartodbpositron",
    zoom_start=2,
)

folium.GeoJson(r"GeospacialDBs_Data/countries.geo.json", name="geojson").add_to(m)

m

In [ ]:
hm = plugins.HeatMapWithTime(
    data=locs_yearly,
    index=date_index,
    auto_play=True,
    radius=4,
    max_opacity=0.3
)


In [ ]:
folium.Choropleth(
    geo_data=r"GeospacialDBs_Data/countries.geo.json",
    name="choropleth",
    data=cases,
    columns=["New_cases", "Country_code"],
    key_on="feature.properties.country_id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)




folium.LayerControl().add_to(m)
m

In [ ]:
type(cases["New_cases"])

## Bokeh

In [ ]:
merged = countries.merge(cases, left_on = 'Country_code', right_on = 'Country_code')

merged

In [ ]:
merged_json = json.loads(merged[0:1000].to_json())

json_data = json.dumps(merged_json)

In [ ]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)
#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'New_cases', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
geosource